<a href="https://colab.research.google.com/github/ds-praveenkumar/truefoundry-projects/blob/dev/feedback_prize_effectiveness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install -q kaggle
!pip install -–upgrade pip 


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -–


In [25]:
!pip --version

pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


In [14]:
# from google.colab import files
# files.upload()

In [15]:
! cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [16]:
!kaggle competitions download -c feedback-prize-effectiveness

feedback-prize-effectiveness.zip: Skipping, found more recently modified local copy (use --force to force download)


In [17]:
!ls -l


total 8340
-rw-r--r-- 1 root root 8528364 Jul  8 04:38 feedback-prize-effectiveness.zip
-rw-r--r-- 1 root root      68 Jul  8 04:35 kaggle.json
drwxr-xr-x 1 root root    4096 Jul  6 13:22 sample_data


In [ ]:
! unzip feedback-prize-effectiveness.zip -d data

In [26]:
 !pip install mlfoundry -q

In [27]:
import mlfoundry
mlfoundry.login()

Please get your API key from https://app.truefoundry.com/settings
Paste your API key and hit enter:··········
Writing API key at /root/.mlfoundry/credentials.netrc


True

In [29]:
client = mlfoundry.get_client()
run = client.create_run(project_name="kaggle", run_name="feedback-prize-effectiveness")


[mlfoundry] 2022-07-08T05:04:42+0000 INFO project kaggle does not exist. Creating kaggle.
Link to the dashboard for the run: https://app.truefoundry.com/mlfoundry/182/a1b2faa30b91463199d2d656f7987026/
[mlfoundry] 2022-07-08T05:04:44+0000 WARNING failed to log git info because git repository is not present
[mlfoundry] 2022-07-08T05:04:45+0000 INFO Run 'prokaggler/kaggle/feedback-prize-effectiveness' has started.
